In [10]:
from langchain.llms.ollama import Ollama
from langchain.chat_models import ChatOllama
from langchain.prompts import PromptTemplate, ChatPromptTemplate , MessagesPlaceholder
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

from langchain.document_loaders import TextLoader , PyPDFLoader , UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory


from langchain_community.embeddings import OllamaEmbeddings

# llm = Ollama(model="deepseek-r1:14b")
# chat = ChatOllama(
#     model="deepseek-r1:14b",

llm = Ollama(model="llama3.1:latest")
chat = ChatOllama(
    model="llama3.1:latest",
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
    
    )

문서 로드와 쪼개기 / 임베딩 생성 및 캐시

In [11]:
cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

# loader = TextLoader("./chapter_one.txt")
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

# embeddings = OpenAIEmbeddings()

ollama_emb = OllamaEmbeddings(
    model="llama3.1:latest",
)


cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    ollama_emb,
    cache_dir,
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

메모리와 문서를 이용한 프롬프트

In [12]:
memory = ConversationBufferMemory(
    llm=llm,
    max_token_limit=20,
    return_messages=True,
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_8104\3804591126.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


체인 연결 / 체인 호출 함수 정의

In [31]:
chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "history": load_memory,
    }
    | prompt
    | llm
)


def invoke_chain(question):
    result = chain.invoke(question)
    # print(question)
    print(result)
    memory.save_context({"input": question}, {"output": result})


체인에 질문하여 테스트

In [32]:
invoke_chain("Victory Mansions")
invoke_chain("Tell me about the characters")

It seems like we've been discussing multiple questions about "Victory Mansions" earlier. I'll provide a more in-depth answer to your current question, which is about the characters mentioned in the passage.

Let's break down the characters:

1. **The Narrator**: The narrator is the main character of the story, although their name is not provided. They are a resident of the city, where they witness the effects of war and totalitarianism.
2. **Goldstein**: Goldstein is a subversive figure who opposes Big Brother's regime. He is the target of the Hate broadcasts and is accused of treason. The narrator has a complex emotional response to Goldstein, feeling a mix of emotions such as fear, anger, and possibly even admiration for his courage.
3. **The Dark-Haired Girl**: The dark-haired girl is a minor character who appears behind the narrator during the Hate broadcast. She wears a scarlet sash, which symbolizes her chastity. The narrator's hatred for her is motivated by his own desires and f

메모리를 출력하여 메모리가 체인에 적용되었는지 확인

In [30]:
load_memory({})

[HumanMessage(content='Victory Mansions', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Victory Mansions is a location mentioned in the text. It appears to be a residential building or complex where the narrator lives or has visited. The exact address or location of Victory Mansions is not specified, but it is described as a place that has been affected by war and destruction.\n\nThe description of Victory Mansions includes:\n\n* Ruins and devastation\n* Buildings with broken windows and crumbling facades\n* Overgrown gardens and unkempt streets\n\nThis suggests that Victory Mansions may be located in a post-war or devastated area, perhaps in the same city or region where the narrator lives.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Victory Mansions', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Victory Mansions is indeed a location mentioned in the text. According to the passage, Victory Mansions is a building that can